In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### SVM 

In [ ]:
import tensorflow as tf

In [ ]:
#dir paths 
train_dir='/kaggle/input/dogs-cats-small-dataset/train'
test_dir='/kaggle/input/dogs-cats-small-dataset/test1'

In [ ]:
#Create a training dataset 
train_dataset=tf.keras.preprocessing.image_dataset_from_directory(
train_dir,
image_size=(64,64),
batch_size=32,
label_mode='binary', #Use Binary labels (0 for cat and 1 for dog)
)

test_dataset=tf.keras.preprocessing.image_dataset_from_directory(
test_dir,
image_size=(64,64),
batch_size=32,
label_mode=None, #Use Binary labels (0 for cat and 1 for dog)
shuffle=False 
)

In [ ]:
def preprocess(image,label):
    image=tf.image.resize(image,(64,64))
    image=tf.cast(image,tf.float32)/255.0
    return image,label

#Apply preprocessing to the datasets
train_dataset=train_dataset.map(preprocess)

#Convert to numpy arrays 
train_images,train_labels=[],[]
for images,labels in train_dataset.unbatch():
    train_images.append(images.numpy().flatten())
    train_labels.append(labels.numpy())
    
train_images=np.array(train_images)
train_labels=np.array(train_labels)

In [ ]:
test_images=[]
for images in test_dataset.unbatch():
    image=tf.image.resize(images,(64,64))
    image=tf.cast(image,tf.float32)/255.0
    test_images.append(image.numpy().flatten())
    
test_images=np.array(test_images)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#Split the data into training and validation set 
train_img,valid_img,train_lab,valid_lab=train_test_split(train_images,
                                                         train_labels,
                                                         test_size=0.2,
                                                         random_state=1)

In [ ]:
#Inistialise the SVM
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,classification_report

svc=SVC(kernel='linear',random_state=1)

svc.fit(train_img,train_lab)

train_pred=svc.predict(train_img)
print(f"Training Accuracy:{accuracy_score(train_lab,train_pred)}")
print("Training Classification Report:")
print(classification_report(train_lab,train_pred))

In [ ]:
valid_pred=svc.predict(valid_img)
print(f"Validation Accuracy:{accuracy_score(valid_lab,valid_pred)}")
print("Validation Classification Report:")
print(classification_report(valid_lab,valid_pred))

In [ ]:
import matplotlib.pyplot as plt

#Function to plot a image
def plot_image(image,true_label,predict_label):
    plt.figure(figsize=(10,10))
    if image.ndim==1:
        image=image.reshape(64,64,3)
    plt.imshow(image,cmap=plt.cm.binary)
    plt.title(f"Predicted Label:{predict_label}\nActual Label:{true_label}")
    plt.axis('off')
    plt.show()
    

image=train_img[0]
train_true_lab=train_lab[0]
train_pred_lab=train_pred[0]

plot_image(image,train_true_lab,train_pred_lab)